1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰 수 초과로 답변을 생성하지 못할 수 있음
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> vector DB 에 저장
4. 질문이 있을 때, vector db 에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [78]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)


In [79]:
document_list
len(document_list)

187

In [100]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [101]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(
#    documents=document_list,
#    embedding=embeddings, 
#    collection_name="chroma-tax",
#    persist_directory="./chroma_db",
#)

database = Chroma(
    collection_name="chroma-tax",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)

In [102]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

In [103]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2")

In [104]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/Users/nhn/Desktop/llm-application/lib/python3.10/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [105]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [107]:
ai_message = qa_chain({"query": query})

In [108]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '고용주가 임금에 대한 상여金과 지급일, 지급일이 있는지 여부 etc.에 의한 근로소득의 변동으로 인한 소득세의 연말정산을 하는 경우가 있다.'}